In [30]:
import os
import glob
import pandas as pd
from tqdm import tqdm
import boto3
from io import BytesIO
from io import StringIO

In [31]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [32]:
# Initialize S3 client
s3 = boto3.client('s3')

# Function to load data 
bucket_name = 'ml-stack-759578648427-us-west-2'

def from_s3(bucket_name, file_name):
    response = s3.get_object(Bucket=bucket_name, Key=file_name)
    ext = os.path.splitext(file_name)[1]
    if ext == '.gz':
        data = pd.read_csv(BytesIO(response['Body'].read()), compression='gzip')
    elif ext == '.csv':
        data = pd.read_csv(BytesIO(response['Body'].read()))
    else:
        data = pd.read_excel(BytesIO(response['Body'].read()), sheet_name='DX_to_CCSR_Mapping', header=1)
    return data

### Baseline

In [33]:
file_name = 'final_microbiology_df.csv.gz'
df = from_s3(bucket_name, file_name) 
#df['charttime'] = pd.to_datetime(df['charttime'])
#subjects = df['subject_id'].unique()
#df.head()

In [34]:
# final pairs 
pairs = [('ESCHERICHIA COLI', 'AMPICILLIN'),
 ('ESCHERICHIA COLI', 'AMPICILLIN/SULBACTAM'),
 ('ESCHERICHIA COLI', 'CEFAZOLIN'),
 ('ESCHERICHIA COLI', 'CEFEPIME'),
 ('ESCHERICHIA COLI', 'CEFTAZIDIME'),
 ('ESCHERICHIA COLI', 'CEFTRIAXONE'),
 ('ESCHERICHIA COLI', 'CIPROFLOXACIN'),
 ('ESCHERICHIA COLI', 'GENTAMICIN'),
 ('ESCHERICHIA COLI', 'MEROPENEM'),
 ('ESCHERICHIA COLI', 'NITROFURANTOIN'),
 ('ESCHERICHIA COLI', 'TOBRAMYCIN'),
 ('ESCHERICHIA COLI', 'TRIMETHOPRIM/SULFA'),
 ('PSEUDOMONAS AERUGINOSA', 'CEFEPIME'),
 ('PSEUDOMONAS AERUGINOSA', 'CEFTAZIDIME'),
 ('PSEUDOMONAS AERUGINOSA', 'CIPROFLOXACIN'),
 ('PSEUDOMONAS AERUGINOSA', 'GENTAMICIN'),
 ('PSEUDOMONAS AERUGINOSA', 'MEROPENEM'),
 ('PSEUDOMONAS AERUGINOSA', 'PIPERACILLIN/TAZO'),
 ('PSEUDOMONAS AERUGINOSA', 'TOBRAMYCIN'),
 ('ESCHERICHIA COLI', 'PIPERACILLIN/TAZO'),
 ('KLEBSIELLA PNEUMONIAE', 'AMPICILLIN/SULBACTAM'),
 ('KLEBSIELLA PNEUMONIAE', 'CEFAZOLIN'),
 ('KLEBSIELLA PNEUMONIAE', 'CEFEPIME'),
 ('KLEBSIELLA PNEUMONIAE', 'CEFTAZIDIME'),
 ('KLEBSIELLA PNEUMONIAE', 'CEFTRIAXONE'),
 ('KLEBSIELLA PNEUMONIAE', 'CIPROFLOXACIN'),
 ('KLEBSIELLA PNEUMONIAE', 'GENTAMICIN'),
 ('KLEBSIELLA PNEUMONIAE', 'MEROPENEM'),
 ('KLEBSIELLA PNEUMONIAE', 'NITROFURANTOIN'),
 ('KLEBSIELLA PNEUMONIAE', 'PIPERACILLIN/TAZO'),
 ('KLEBSIELLA PNEUMONIAE', 'TOBRAMYCIN'),
 ('KLEBSIELLA PNEUMONIAE', 'TRIMETHOPRIM/SULFA')]

df = df[df[['org_name', 'ab_name']].apply(tuple, axis=1).isin(pairs)]

In [35]:
# get unique org & ab combos 
org_ab_label = df[['org_name', 'ab_name']].drop_duplicates()

In [36]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

# for each label calculate metrics for majority classifier 
baselines = []

for index, row in org_ab_label.iterrows():
    org = f"{row.org_name}"
    ab = f"{row.ab_name}"
    sub = df.query(f"org_name == '{org}' & ab_name == '{ab}'")
    y_true = sub.susceptible_flag 

    if (y_true.sum() / len(y_true)) > 0.5:
        maj_class = 1
    else:
        maj_class = 0
    
    y_pred = [maj_class] * len(y_true)
    
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    metrics = {'bacteria':org, 'antibiotic':ab, 'accuracy':accuracy, 'precision':precision, 'recall':recall, 'f1_score':f1}
    baselines.append(metrics)

/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [37]:
baseline_results = pd.DataFrame(baselines)
baseline_results

,bacteria,antibiotic,accuracy,precision,recall,f1_score
0,ESCHERICHIA COLI,AMPICILLIN,0.503929,0.503929,1.0,0.670150
1,ESCHERICHIA COLI,AMPICILLIN/SULBACTAM,0.611413,0.611413,1.0,0.758853
2,ESCHERICHIA COLI,CEFAZOLIN,0.805609,0.805609,1.0,0.892341
3,ESCHERICHIA COLI,CEFEPIME,0.915470,0.915470,1.0,0.955870
4,ESCHERICHIA COLI,CEFTAZIDIME,0.921242,0.921242,1.0,0.959007
5,ESCHERICHIA COLI,CEFTRIAXONE,0.882644,0.882644,1.0,0.937664
6,ESCHERICHIA COLI,CIPROFLOXACIN,0.732396,0.732396,1.0,0.845530
7,ESCHERICHIA COLI,GENTAMICIN,0.897404,0.897404,1.0,0.945928
8,ESCHERICHIA COLI,MEROPENEM,0.998653,0.998653,1.0,0.999326
9,ESCHERICHIA COLI,NITROFURANTOIN,0.956875,0.956875,1.0,0.977962


In [38]:
s3_key = 'MIMIC/Modeling/baseline_results.csv'

# Save the DataFrame to a CSV in memory (using StringIO)
csv_buffer = StringIO()
baseline_results.to_csv(csv_buffer, index=False)

# Upload the CSV data to S3
s3.put_object(Bucket=bucket_name, Key=s3_key, Body=csv_buffer.getvalue())

print(f"DataFrame successfully exported to s3://{bucket_name}/{s3_key}")

DataFrame successfully exported to s3://ml-stack-759578648427-us-west-2/MIMIC/Modeling/baseline_results.csv


### Imports

In [39]:
# Brian's data 
file_name = 'MIMIC/Cleaned/targets_w_lab_features_v3a.csv.gz'
df = from_s3(bucket_name, file_name) 

# katt's data 
file_name = 'MIMIC/Cleaned/df_omr_pt_ad_diag_proc_pr.csv'
omr_df = from_s3(bucket_name, file_name) 

In [40]:
print(df.shape)
print(omr_df.shape)

(504738, 54)
(35529, 978)


### Merge

In [41]:
df = df.merge(omr_df, 
              on=["subject_id","charttime"], 
              how="left")

In [42]:
df.shape

(504738, 1030)

In [43]:
# Count NaN values in each of the new lab test columns
nan_counts = df.isna().sum()

# Display the result
print(nan_counts)
print(df.shape[0])

subject_id                                      0
org_name                                        0
ab_name                                         0
charttime                                       0
interpretation                                  0
susceptible_flag                                0
charttime_target                                0
Blood - Hematocrit                          48598
Blood - Platelet Count                      51996
Blood - MCV                                 47240
Blood - Red Blood Cells                     41891
Blood - RDW                                 41876
Blood - MCH                                 41828
Blood - MCHC                                41826
Blood - White Blood Cells                   50783
Blood - Creatinine                          50738
Blood - Urea Nitrogen                       47044
Blood - Potassium                          121961
Blood - Sodium                             110031
Blood - Chloride                            55369


### Feature Selection

In [44]:
omr_features = omr_df.columns.tolist()
drop_from_omr = [
    'subject_id',
    'charttime',
    'Height (Inches)', #
    'Weight (Lbs)', #
    'bp_systolic', #
    'bp_diastolic', #
    'gender', #
    'anchor_age', #
    'insurance', #
    'language', #
    'marital_status', #
    'race', #
    'procedure_ct', #
    'days_since_last_proc' #
]

omr_features = [col for col in omr_features if col not in drop_from_omr]

In [45]:
ids = ['subject_id','charttime']

pop = ['org_name','ab_name']

target = ['susceptible_flag']

numeric_features = [
    "Blood - Hematocrit",
    "Blood - Platelet Count",
    "Blood - MCV",
    "Blood - Red Blood Cells",
    "Blood - RDW",
    "Blood - MCH",
    "Blood - MCHC",
    "Blood - White Blood Cells",
    "Blood - Creatinine",
    "Blood - Urea Nitrogen",
    "Blood - Potassium",
    "Blood - Sodium",
    "Blood - Chloride",
    "Blood - Bicarbonate",
    "Blood - Anion Gap",
    "Blood - Hemoglobin",
    'Blood - Basophils',
    'Blood - Lymphocytes',
    'Blood - Eosinophils',    
    'Blood - Calcium, Total',
    'Blood - Alanine Aminotransferase (ALT)',
    'Blood - Asparate Aminotransferase (AST)', 
    'Blood - Magnesium',
    'Blood - Phosphate',
    'Blood - INR(PT)',
    'Blood - PT',
    'Blood - Alkaline Phosphatase',
    'Blood - Bilirubin, Total',
#    'Blood - Albumin',
#    'Blood - RDW-SD',
    'Blood - Neutrophils',
#    'Blood - Immature Granulocytes',
#    'Blood - Lactate',
    'Blood - PTT',
    'Urine - Specific Gravity',
    'Urine - pH',
    'anchor_age',
    'Height (Inches)',
    'Weight (Lbs)',
    'bp_systolic',
    'bp_diastolic',
    'procedure_ct',
    'days_since_last_proc'
]


cat_features = [
    "Urine - Leukocytes",
    "Urine - Nitrite",
    "Urine - Bilirubin",
    "Urine - Blood",
    "Urine - Yeast",
    'gender', 
    'insurance', 
    'language',
    'marital_status',
    'race'    
]
flag_features = [
    'Blood - Glucose',
    'Blood - Monocytes',
    'Urine - Urine Color',
    'Urine - Urine Appearance',
    'Urine - Ketone',
    'Urine - Glucose',
    'Urine - Protein',
    'Urine - Epithelial Cells',
    'Urine - RBC',
    'Urine - WBC']

In [46]:
selected_cols = ids+ pop + target + numeric_features + cat_features + flag_features + omr_features
# Filter dataframe
df_filtered = df[selected_cols]

# Display the shape of the new dataframe
print(df_filtered.shape)

(504738, 1028)


### Population Selection

In [47]:
# Drop all NAs
df_filtered2 = df_filtered.dropna()

print(df_filtered2.shape)
df_filtered2.head()

(78304, 1028)


,subject_id,charttime,org_name,ab_name,susceptible_flag,Blood - Hematocrit,Blood - Platelet Count,Blood - MCV,Blood - Red Blood Cells,Blood - RDW,Blood - MCH,Blood - MCHC,Blood - White Blood Cells,Blood - Creatinine,Blood - Urea Nitrogen,Blood - Potassium,Blood - Sodium,Blood - Chloride,Blood - Bicarbonate,Blood - Anion Gap,Blood - Hemoglobin,Blood - Basophils,Blood - Lymphocytes,Blood - Eosinophils,"Blood - Calcium, Total",Blood - Alanine Aminotransferase (ALT),Blood - Asparate Aminotransferase (AST),Blood - Magnesium,Blood - Phosphate,Blood - INR(PT),Blood - PT,Blood - Alkaline Phosphatase,"Blood - Bilirubin, Total",Blood - Neutrophils,Blood - PTT,Urine - Specific Gravity,Urine - pH,anchor_age,Height (Inches),Weight (Lbs),bp_systolic,bp_diastolic,procedure_ct,days_since_last_proc,Urine - Leukocytes,Urine - Nitrite,Urine - Bilirubin,Urine - Blood,Urine - Yeast,gender,insurance,language,marital_status,race,Blood - Glucose,Blood - Monocytes,Urine - Urine Color,Urine - Urine Appearance,Urine - Ketone,Urine - Glucose,Urine - Protein,Urine - Epithelial Cells,Urine - RBC,Urine - WBC,99,BLD001,BLD002,BLD003,BLD004,BLD005,BLD006,BLD007,BLD008,BLD009,BLD010,CIR001,CIR002,CIR003,CIR004,CIR005,CIR006,CIR007,CIR008,CIR009,CIR010,CIR011,CIR012,CIR013,CIR014,CIR015,CIR016,CIR017,CIR018,CIR019,CIR020,CIR021,CIR022,CIR023,CIR024,CIR025,CIR026,CIR027,CIR028,CIR029,CIR030,CIR031,CIR032,CIR033,CIR034,CIR035,CIR036,CIR037,CIR038,CIR039,DEN001,DEN002,DEN003,DIG001,DIG002,DIG003,DIG004,DIG005,DIG006,DIG007,DIG008,DIG009,DIG010,DIG011,DIG012,DIG013,DIG014,DIG015,DIG016,DIG017,DIG018,DIG019,DIG020,DIG021,DIG022,DIG023,DIG024,DIG025,EAR001,EAR002,EAR003,EAR004,EAR005,EAR006,END001,END002,END003,END004,END005,END006,END007,END008,END009,END010,END011,END012,END013,END014,END015,END016,END017,EXT001,EXT002,EXT003,EXT004,EXT005,EXT006,EXT007,EXT008,EXT009,EXT010,EXT011,EXT012,EXT013,EXT014,EXT015,EXT016,EXT017,EXT018,EXT019,EXT020,EXT021,EXT022,EXT023,EXT024,EXT025,EXT026,EXT027,EXT028,EXT029,EXT030,EYE001,EYE002,EYE003,EYE004,EYE005,EYE006,EYE007,EYE008,EYE009,EYE010,EYE011,EYE012,FAC001,FAC002,FAC003,FAC004,FAC005,FAC006,FAC008,FAC009,FAC010,FAC012,FAC013,FAC014,FAC015,FAC016,FAC017,FAC019,FAC020,FAC022,FAC023,FAC024,FAC025,FAC026,FAC027,FAC028,FAC029,FAC030,GEN001,GEN002,GEN003,GEN004,GEN005,GEN006,GEN007,GEN008,GEN009,GEN010,GEN011,GEN012,GEN013,GEN014,GEN015,GEN016,GEN017,GEN018,GEN019,GEN020,GEN021,GEN022,GEN023,GEN024,GEN025,GEN026,INF001,INF002,INF003,INF004,INF005,INF006,INF007,INF008,INF009,INF010,INF011,INF012,INJ001,INJ002,INJ003,INJ004,INJ005,INJ006,INJ007,INJ008,INJ009,INJ010,INJ011,INJ012,INJ013,INJ014,INJ015,INJ016,INJ017,INJ018,INJ019,INJ020,INJ021,INJ022,INJ023,INJ024,INJ025,INJ026,INJ027,INJ028,INJ029,INJ030,INJ031,INJ032,INJ033,INJ034,INJ035,INJ036,INJ037,INJ038,INJ039,INJ040,INJ041,INJ042,INJ043,INJ044,INJ045,INJ046,INJ047,INJ048,INJ049,INJ050,INJ054,INJ055,INJ056,INJ057,INJ059,INJ060,INJ061,INJ062,INJ063,INJ064,INJ065,INJ066,INJ067,INJ069,INJ070,INJ071,INJ072,INJ073,INJ074,INJ075,INJ076,MAL001,MAL002,MAL003,MAL004,MAL005,MAL006,MAL007,MAL008,MAL009,MAL010,MBD001,MBD002,MBD003,MBD004,MBD005,MBD006,MBD007,MBD008,MBD009,MBD010,MBD011,MBD012,MBD013,MBD014,MBD017,MBD018,MBD019,MBD020,MBD021,MBD022,MBD023,MBD024,MBD025,MBD026,MBD027,MBD028,MBD032,MBD034,MUS001,MUS002,MUS003,MUS004,MUS005,MUS006,MUS007,MUS008,MUS009,MUS010,MUS011,MUS012,MUS013,MUS014,MUS015,MUS016,MUS017,MUS020,MUS021,MUS022,MUS023,MUS024,MUS025,MUS026,MUS027,MUS028,MUS029,MUS030,MUS031,MUS032,MUS033,MUS034,MUS035,MUS036,MUS037,MUS038,NEO001,NEO002,NEO003,NEO004,NEO005,NEO006,NEO007,NEO008,NEO009,NEO010,NEO011,NEO012,NEO013,NEO014,NEO015,NEO016,NEO017,NEO018,NEO019,NEO020,NEO021,NEO022,NEO023,NEO024,NEO025,NEO026,NEO027,NEO028,NEO029,NEO030,NEO031,NEO032,NEO033,NEO034,NEO035,NEO036,NEO037,NEO038,NEO039,NEO040,NEO041,NEO042,NEO043,NEO044,NEO045,NEO046,NEO047,NEO048,NEO049,NEO050,NEO051,NEO052,NEO054,NEO056,NEO057,NEO058,NEO059,NEO060,NEO061,NEO062,NEO063,NEO064,NEO065,NEO0

In [48]:
# Look at before and after
df_grouped1 = df_filtered.groupby(['org_name', 'ab_name']).size().reset_index(name='count')
df_grouped2 = df_filtered2.groupby(['org_name', 'ab_name']).size().reset_index(name='count')

# Analysis
df_grouped3 = df_grouped1.merge(df_grouped2,
                                on = ['org_name','ab_name'],
                                how='left',
                                suffixes=('_starting', '_final'))

df_grouped3['difference'] = df_grouped3['count_starting'] - df_grouped3['count_final']
df_grouped3.sort_values(by = 'count_final', ascending=False, inplace=True)
df_grouped3

,org_name,ab_name,count_starting,count_final,difference
22,ESCHERICHIA COLI,TRIMETHOPRIM/SULFA,27437,3668.0,23769.0
15,ESCHERICHIA COLI,CIPROFLOXACIN,27466,3665.0,23801.0
10,ESCHERICHIA COLI,AMPICILLIN/SULBACTAM,27461,3665.0,23796.0
16,ESCHERICHIA COLI,GENTAMICIN,27467,3665.0,23802.0
21,ESCHERICHIA COLI,TOBRAMYCIN,27464,3665.0,23799.0
11,ESCHERICHIA COLI,CEFAZOLIN,27455,3664.0,23791.0
13,ESCHERICHIA COLI,CEFTAZIDIME,27464,3663.0,23801.0
17,ESCHERICHIA COLI,MEROPENEM,27460,3662.0,23798.0
14,ESCHERICHIA COLI,CEFTRIAXONE,27455,3662.0,23793.0
9,ESCHERICHIA COLI,AMPICILLIN,27359,3653.0,23706.0


In [49]:
# Export results
# df_grouped3.to_excel("../mimic-iv-3.1/modelling_population_NAN.xlsx", index=False)

In [50]:
# Filter out those with less than 100
to_remove = df_grouped2[df_grouped2['count'] < 100]

# Perform an anti-join to remove these combinations
df_filtered3 = df_filtered2.merge(to_remove, on=['org_name', 'ab_name'], how='left', indicator=True)

# Keep only the rows that are not in the low-count combinations
df_filtered3 = df_filtered3[df_filtered3['_merge'] == 'left_only'].drop(columns=['_merge'])
df_filtered3.drop(columns=['count'],inplace=True)
df_filtered3.shape

(78124, 1028)

In [51]:
# Check
df_filtered3.groupby(['org_name', 'ab_name']).size().reset_index(name='count')

,org_name,ab_name,count
0,ENTEROCOCCUS FAECALIS,AMPICILLIN,495
1,ENTEROCOCCUS FAECALIS,DAPTOMYCIN,190
2,ENTEROCOCCUS FAECALIS,NITROFURANTOIN,235
3,ENTEROCOCCUS FAECALIS,PENICILLIN G,267
4,ENTEROCOCCUS FAECALIS,TETRACYCLINE,255
5,ENTEROCOCCUS FAECALIS,VANCOMYCIN,494
6,ESCHERICHIA COLI,AMPICILLIN,3653
7,ESCHERICHIA COLI,AMPICILLIN/SULBACTAM,3665
8,ESCHERICHIA COLI,CEFAZOLIN,3664
9,ESCHERICHIA COLI,CEFEPIME,3642


In [52]:
final_combinations = df_filtered3[['org_name', 'ab_name']].drop_duplicates()
# Remove E Facaelis
final_combinations = final_combinations[final_combinations['org_name']!='ENTEROCOCCUS FAECALIS']
bacteria_antibiotic_pairs = list(final_combinations.itertuples(index=False, name=None))
bacteria_antibiotic_pairs

[('ESCHERICHIA COLI', 'AMPICILLIN'),
 ('ESCHERICHIA COLI', 'AMPICILLIN/SULBACTAM'),
 ('ESCHERICHIA COLI', 'CEFAZOLIN'),
 ('ESCHERICHIA COLI', 'CEFEPIME'),
 ('ESCHERICHIA COLI', 'CEFTAZIDIME'),
 ('ESCHERICHIA COLI', 'CEFTRIAXONE'),
 ('ESCHERICHIA COLI', 'CIPROFLOXACIN'),
 ('ESCHERICHIA COLI', 'GENTAMICIN'),
 ('ESCHERICHIA COLI', 'MEROPENEM'),
 ('ESCHERICHIA COLI', 'NITROFURANTOIN'),
 ('ESCHERICHIA COLI', 'PIPERACILLIN/TAZO'),
 ('ESCHERICHIA COLI', 'TOBRAMYCIN'),
 ('ESCHERICHIA COLI', 'TRIMETHOPRIM/SULFA'),
 ('KLEBSIELLA PNEUMONIAE', 'AMPICILLIN/SULBACTAM'),
 ('KLEBSIELLA PNEUMONIAE', 'CEFAZOLIN'),
 ('KLEBSIELLA PNEUMONIAE', 'CEFEPIME'),
 ('KLEBSIELLA PNEUMONIAE', 'CEFTAZIDIME'),
 ('KLEBSIELLA PNEUMONIAE', 'CEFTRIAXONE'),
 ('KLEBSIELLA PNEUMONIAE', 'CIPROFLOXACIN'),
 ('KLEBSIELLA PNEUMONIAE', 'GENTAMICIN'),
 ('KLEBSIELLA PNEUMONIAE', 'MEROPENEM'),
 ('KLEBSIELLA PNEUMONIAE', 'NITROFURANTOIN'),
 ('KLEBSIELLA PNEUMONIAE', 'PIPERACILLIN/TAZO'),
 ('KLEBSIELLA PNEUMONIAE', 'TOBRAMYCIN'),
 ('KL

In [53]:
# Ecoli only
ecoli_combinations = final_combinations[final_combinations['org_name']=='ESCHERICHIA COLI']
ecoli_antibiotic_pairs = list(ecoli_combinations.itertuples(index=False, name=None))
ecoli_antibiotic_pairs

[('ESCHERICHIA COLI', 'AMPICILLIN'),
 ('ESCHERICHIA COLI', 'AMPICILLIN/SULBACTAM'),
 ('ESCHERICHIA COLI', 'CEFAZOLIN'),
 ('ESCHERICHIA COLI', 'CEFEPIME'),
 ('ESCHERICHIA COLI', 'CEFTAZIDIME'),
 ('ESCHERICHIA COLI', 'CEFTRIAXONE'),
 ('ESCHERICHIA COLI', 'CIPROFLOXACIN'),
 ('ESCHERICHIA COLI', 'GENTAMICIN'),
 ('ESCHERICHIA COLI', 'MEROPENEM'),
 ('ESCHERICHIA COLI', 'NITROFURANTOIN'),
 ('ESCHERICHIA COLI', 'PIPERACILLIN/TAZO'),
 ('ESCHERICHIA COLI', 'TOBRAMYCIN'),
 ('ESCHERICHIA COLI', 'TRIMETHOPRIM/SULFA')]

### OHE

In [54]:
from sklearn.preprocessing import OneHotEncoder

# Initialize encoder
encoder = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')

# Fit and transform categorical data
encoded_array = encoder.fit_transform(df_filtered3[cat_features])

# Convert to DataFrame
encoded_df = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out(cat_features))

# Concatenate with original dataframe (dropping old categorical columns)
df_final = pd.concat([df_filtered3.drop(columns=cat_features), encoded_df], axis=1)

# Display new dataframe shape
print(df_final.shape)

# Check new columns
df_final.head(5)


(78304, 1117)


,subject_id,charttime,org_name,ab_name,susceptible_flag,Blood - Hematocrit,Blood - Platelet Count,Blood - MCV,Blood - Red Blood Cells,Blood - RDW,Blood - MCH,Blood - MCHC,Blood - White Blood Cells,Blood - Creatinine,Blood - Urea Nitrogen,Blood - Potassium,Blood - Sodium,Blood - Chloride,Blood - Bicarbonate,Blood - Anion Gap,Blood - Hemoglobin,Blood - Basophils,Blood - Lymphocytes,Blood - Eosinophils,"Blood - Calcium, Total",Blood - Alanine Aminotransferase (ALT),Blood - Asparate Aminotransferase (AST),Blood - Magnesium,Blood - Phosphate,Blood - INR(PT),Blood - PT,Blood - Alkaline Phosphatase,"Blood - Bilirubin, Total",Blood - Neutrophils,Blood - PTT,Urine - Specific Gravity,Urine - pH,anchor_age,Height (Inches),Weight (Lbs),bp_systolic,bp_diastolic,procedure_ct,days_since_last_proc,Blood - Glucose,Blood - Monocytes,Urine - Urine Color,Urine - Urine Appearance,Urine - Ketone,Urine - Glucose,Urine - Protein,Urine - Epithelial Cells,Urine - RBC,Urine - WBC,99,BLD001,BLD002,BLD003,BLD004,BLD005,BLD006,BLD007,BLD008,BLD009,BLD010,CIR001,CIR002,CIR003,CIR004,CIR005,CIR006,CIR007,CIR008,CIR009,CIR010,CIR011,CIR012,CIR013,CIR014,CIR015,CIR016,CIR017,CIR018,CIR019,CIR020,CIR021,CIR022,CIR023,CIR024,CIR025,CIR026,CIR027,CIR028,CIR029,CIR030,CIR031,CIR032,CIR033,CIR034,CIR035,CIR036,CIR037,CIR038,CIR039,DEN001,DEN002,DEN003,DIG001,DIG002,DIG003,DIG004,DIG005,DIG006,DIG007,DIG008,DIG009,DIG010,DIG011,DIG012,DIG013,DIG014,DIG015,DIG016,DIG017,DIG018,DIG019,DIG020,DIG021,DIG022,DIG023,DIG024,DIG025,EAR001,EAR002,EAR003,EAR004,EAR005,EAR006,END001,END002,END003,END004,END005,END006,END007,END008,END009,END010,END011,END012,END013,END014,END015,END016,END017,EXT001,EXT002,EXT003,EXT004,EXT005,EXT006,EXT007,EXT008,EXT009,EXT010,EXT011,EXT012,EXT013,EXT014,EXT015,EXT016,EXT017,EXT018,EXT019,EXT020,EXT021,EXT022,EXT023,EXT024,EXT025,EXT026,EXT027,EXT028,EXT029,EXT030,EYE001,EYE002,EYE003,EYE004,EYE005,EYE006,EYE007,EYE008,EYE009,EYE010,EYE011,EYE012,FAC001,FAC002,FAC003,FAC004,FAC005,FAC006,FAC008,FAC009,FAC010,FAC012,FAC013,FAC014,FAC015,FAC016,FAC017,FAC019,FAC020,FAC022,FAC023,FAC024,FAC025,FAC026,FAC027,FAC028,FAC029,FAC030,GEN001,GEN002,GEN003,GEN004,GEN005,GEN006,GEN007,GEN008,GEN009,GEN010,GEN011,GEN012,GEN013,GEN014,GEN015,GEN016,GEN017,GEN018,GEN019,GEN020,GEN021,GEN022,GEN023,GEN024,GEN025,GEN026,INF001,INF002,INF003,INF004,INF005,INF006,INF007,INF008,INF009,INF010,INF011,INF012,INJ001,INJ002,INJ003,INJ004,INJ005,INJ006,INJ007,INJ008,INJ009,INJ010,INJ011,INJ012,INJ013,INJ014,INJ015,INJ016,INJ017,INJ018,INJ019,INJ020,INJ021,INJ022,INJ023,INJ024,INJ025,INJ026,INJ027,INJ028,INJ029,INJ030,INJ031,INJ032,INJ033,INJ034,INJ035,INJ036,INJ037,INJ038,INJ039,INJ040,INJ041,INJ042,INJ043,INJ044,INJ045,INJ046,INJ047,INJ048,INJ049,INJ050,INJ054,INJ055,INJ056,INJ057,INJ059,INJ060,INJ061,INJ062,INJ063,INJ064,INJ065,INJ066,INJ067,INJ069,INJ070,INJ071,INJ072,INJ073,INJ074,INJ075,INJ076,MAL001,MAL002,MAL003,MAL004,MAL005,MAL006,MAL007,MAL008,MAL009,MAL010,MBD001,MBD002,MBD003,MBD004,MBD005,MBD006,MBD007,MBD008,MBD009,MBD010,MBD011,MBD012,MBD013,MBD014,MBD017,MBD018,MBD019,MBD020,MBD021,MBD022,MBD023,MBD024,MBD025,MBD026,MBD027,MBD028,MBD032,MBD034,MUS001,MUS002,MUS003,MUS004,MUS005,MUS006,MUS007,MUS008,MUS009,MUS010,MUS011,MUS012,MUS013,MUS014,MUS015,MUS016,MUS017,MUS020,MUS021,MUS022,MUS023,MUS024,MUS025,MUS026,MUS027,MUS028,MUS029,MUS030,MUS031,MUS032,MUS033,MUS034,MUS035,MUS036,MUS037,MUS038,NEO001,NEO002,NEO003,NEO004,NEO005,NEO006,NEO007,NEO008,NEO009,NEO010,NEO011,NEO012,NEO013,NEO014,NEO015,NEO016,NEO017,NEO018,NEO019,NEO020,NEO021,NEO022,NEO023,NEO024,NEO025,NEO026,NEO027,NEO028,NEO029,NEO030,NEO031,NEO032,NEO033,NEO034,NEO035,NEO036,NEO037,NEO038,NEO039,NEO040,NEO041,NEO042,NEO043,NEO044,NEO045,NEO046,NEO047,NEO048,NEO049,NEO050,NEO051,NEO052,NEO054,NEO056,NEO057,NEO058,NEO059,NEO060,NEO061,NEO062,NEO063,NEO064,NEO065,NEO066,NEO067,NEO068,NEO069,NEO070,NEO071,NEO072,NEO073,NEO074,NEO075,NEO078,NVS001,NVS002,NVS003,NVS004,NVS005,NVS006,NVS007,NVS00

In [55]:
# export final df 
s3_key = 'MIMIC/Cleaned/final_modeling_dataset.csv'

# Save the DataFrame to a CSV in memory (using StringIO)
csv_buffer = StringIO()
df_final.to_csv(csv_buffer, index=False)

# Upload the CSV data to S3
s3.put_object(Bucket=bucket_name, Key=s3_key, Body=csv_buffer.getvalue())

print(f"DataFrame successfully exported to s3://{bucket_name}/{s3_key}")

DataFrame successfully exported to s3://ml-stack-759578648427-us-west-2/MIMIC/Cleaned/final_modeling_dataset.csv
